In [1]:
import pandas as pd
import numpy as np
import common as cm

This exercise consists of 3 parts. Finish the first part to get a mark of 3.0. The first two parts to get 4.0. Finish all exercies to get 5.0.

# Part 1: Concordance

1.1) Given are the following modes of transport: bus, bike, car, train. Each mode is characterized by 2 cost-type criteria: price and time; and 2 gain-type criteria: comfort and reliability.

Mode of transport | Time | Comfort | Price | Reliability
--- | --- | --- | --- | ---
 **bus**  | 6 | 3  | 6 | 2
 **bike** | 8 | 2  | 2 | 8
 **car**  | 2 | 10 | 9 | 7
 **train**| 3 | 5  | 5 | 6
 **by foot**| 10 | 2  | 0 | 5

In [2]:
data = {
    'mode': ['bus', 'bike', 'car', 'train', 'foot'],
    'time': [6, 8, 2, 3, 10],
    'comfort': [3, 2, 10, 5, 2],
    'price': [6, 2, 9, 5, 0],
    'reliability': [2, 8, 7, 6, 5]
}

criteria = ['time', 'comfort', 'price', 'reliability']
data = pd.DataFrame(data)

data

,mode,time,comfort,price,reliability
0,bus,6,3,6,2
1,bike,8,2,2,8
2,car,2,10,9,7
3,train,3,5,5,6
4,foot,10,2,0,5


1.2)  Below are the parameters, i.e., threholds, criterion-type, and weights, for each criterion,

In [3]:
parameters = {'time': {'weights': 4, 'q': 1.0, 'p': 2, 'v': 4, 'type': 'cost'},
 'comfort': {'weights': 2, 'q': 2.0, 'p': 3, 'v': 6, 'type': 'gain'},
 'price': {'weights': 3, 'q': 1.0, 'p': 3, 'v': 5, 'type': 'cost'},
 'reliability': {'weights': 1, 'q': 1.5, 'p': 3, 'v': 5, 'type': 'gain'}}
sum_weights = 10.
print(type(parameters))
pd.DataFrame(parameters).T

<class 'dict'>


,weights,q,p,v,type
time,4,1,2,4,cost
comfort,2,2,3,6,gain
price,3,1,3,5,cost
reliability,1,1.5,3,5,gain


1.3) Finish the below function for calculating a marginal concordance for $i$-th criterion (gain type) $c_i(g_i(a),g_i(b))$ based on q and p tresholds. 

In [4]:
def getConcordanceGain(gA, gB, q, p):
    gA,gB,q,p = int(gA),int(gB),int(q),int(p)
    if (gA - gB) <= -p:
        return 0
    elif (gA - gB) >= -q:
        return 1
    else:
        return (p-(gB - gA))/(p-q)
    
def getConcordanceCost(gA, gB, q, p):
    return getConcordanceGain(gB, gA, q, p)

1.4)  Calculate  comprehensive concordance  index  for  all  criteria  of  alternatives $a$ and $b$. Remember that comprehensive concordance must be divided by the sum of weights.

In [5]:
def getComprehensiveConcordance(A, B, criteria, parameters):
    concordance = 0.0
    allweights = 0
    lst,lst2 = [],[]
    i = 0
    for a,b in zip(A[1:],B[1:]):
        allweights += parameters[criteria[i]]['weights']
        if parameters[criteria[i]]['type'] == 'gain':
#             print('gain',a,b,getConcordanceGain(a,b,parameters[criteria[i]]['q'],parameters[criteria[i]]['p']))
            lst.append(getConcordanceGain(a,b,parameters[criteria[i]]['q'],parameters[criteria[i]]['p']))
        if parameters[criteria[i]]['type'] == 'cost':
#             print('cost',a,b,getConcordanceCost(a,b,parameters[criteria[i]]['q'],parameters[criteria[i]]['p']))
            lst.append(getConcordanceCost(a,b,parameters[criteria[i]]['q'],parameters[criteria[i]]['p']))
        if i != 4:
            i +=1
    for i,attribute in zip(lst,criteria):
        lst2.append(i*parameters[attribute]['weights'])
    concordance += sum(lst2)
    return concordance/allweights

1.5) Check comprehensive concordance indexes for C(bus, some transportation):

In [6]:
for x in range(5):
    for alternative_id, alternative_row in data.iterrows():
        print("C({0},{1}) = ".format(x, alternative_id), getComprehensiveConcordance(data.loc[x], alternative_row, criteria, parameters))
    print('')

C(0,0) =  1.0
C(0,1) =  0.6
C(0,2) =  0.3
C(0,3) =  0.5
C(0,4) =  0.6

C(1,0) =  0.6
C(1,1) =  1.0
C(1,2) =  0.4
C(1,3) =  0.4
C(1,4) =  0.85

C(2,0) =  0.7
C(2,1) =  0.7
C(2,2) =  1.0
C(2,3) =  0.7
C(2,4) =  0.7

C(3,0) =  1.0
C(3,1) =  0.65
C(3,2) =  0.8
C(3,3) =  1.0
C(3,4) =  0.7

C(4,0) =  0.6
C(4,1) =  0.5
C(4,2) =  0.35
C(4,3) =  0.4
C(4,4) =  1.0



1.6) Finish the below function for generating a concordance matrix. Use a majority_threshold as a cutting-level. For hich pairs a concordance is fulfilled?

In [7]:
def getConcordanceMatrix(data, criteria, parameters, majority_treshold=0.7):
    concordance_matrix = np.zeros((len(data),len(data)))
    for A_idx, A_row in data.iterrows():
        for B_idx, B_row in data.iterrows():
            if A_idx != B_idx:
                if(getComprehensiveConcordance(A_row,B_row,criteria, parameters)>=majority_treshold):
                    concordance_matrix[A_idx][B_idx] = 1
                else:
                    concordance_matrix[A_idx][B_idx] = 0
    return concordance_matrix

In [8]:
print(getConcordanceMatrix(data, criteria, parameters))

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 1. 0. 1. 1.]
 [1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0.]]


# Part 2: outranking graph

2.1) Complete the function that calculates marginal discordance for  $i$-th  criterion (gain type)  $d_i(a,b)$  basing on v treshold. $d_i(a,b) = 1$ when $a$ is worse than $b$ on criterion $i$ by at least the veto threshold, zero otherwise.

In [9]:
def getDiscordanceGain(A ,B, v):
    A,B,v = int(A),int(B),int(v)
    if (B - A) >= v:
        return 1
    return 0

def getDiscordanceCost(A, B, v):
    return getDiscordanceGain(B, A, v)


2.2) Calculate a comprehensive discordance index.  $𝐷(a,b)=1$  if at least one criterion vetoes against $a S b$.

In [10]:
def getComprehensiveDiscordance(alternative_a, alternative_b, criteria, parameters): 
    concordance = 0.0
    lst,lst2 = [],[]
    i = 0
    for a,b in zip(alternative_a[1:],alternative_b[1:]):
        if parameters[criteria[i]]['type'] == 'gain':
            lst.append(getDiscordanceGain(a,b,parameters[criteria[i]]['v']))
        if parameters[criteria[i]]['type'] == 'cost':
            lst.append(getDiscordanceGain(a,b,parameters[criteria[i]]['v']))
        if i != 4:
            i +=1
    concordance +=1 if sum(lst) >= 1 else 0
    return concordance

2.3) Check comprehensive discordance indexes for D(bus, some transportation):

In [11]:
for alternative_id, alternative_row in data.iterrows():
    print("D({0},{1}) = ".format(0, alternative_id),getComprehensiveDiscordance(data.loc[0], alternative_row, criteria, parameters))

D(0,0) =  0.0
D(0,1) =  1.0
D(0,2) =  1.0
D(0,3) =  0.0
D(0,4) =  1.0


2.4) Finish the below function for calculating a comprehensive discordance matrix.

In [29]:
def getDiscordanceMatrix(data, criteria, parameters):
    n = len(data)
    discordance_matrix = np.zeros((n,n))
    for i in range(5):
        for alternative_id, alternative_row in data.iterrows():
            discordance_matrix[i][alternative_id] = getComprehensiveDiscordance(data.loc[i], alternative_row, criteria, parameters)
    return discordance_matrix

In [30]:
getDiscordanceMatrix(data, criteria, parameters)

array([[0., 1., 1., 0., 1.],
       [0., 0., 1., 0., 0.],
       [1., 1., 0., 0., 1.],
       [0., 1., 0., 0., 1.],
       [1., 0., 1., 1., 0.]])

2.5) Now, finish the below function for generating the outranking matrixThis method should take into account both the concordance and discordance matrices.

In [31]:
def getOutrankingMatrix(data, criteria, parameters, majority_treshold):
    n = len(data)
    outranking_matrix = np.zeros((n,n))
    m = 0
    for a,b in zip(getConcordanceMatrix(data, criteria, parameters,majority_treshold),getDiscordanceMatrix(data, criteria, parameters)):
        k = 0
        for i,j in zip(a,b):
            if i == 1.0 and j == 0.0:
                outranking_matrix[m][k] = 1
            elif i == 0.0 or j == 1.0:
                outranking_matrix[m][k] = 0
            k+=1
        m+=1 
    
    return outranking_matrix

In [32]:
outranking_matrix = getOutrankingMatrix(data, criteria, parameters, majority_treshold=0.75)
print(outranking_matrix)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0.]]


2.3) Change outranking matrix to adjacency list as a dictionary that every alternative $a$ has a list of alternatives that $a$ outranks. For simplicity, assume that there are no edges between the vertex and itself (a->a).

In [33]:
data

,mode,time,comfort,price,reliability
0,bus,6,3,6,2
1,bike,8,2,2,8
2,car,2,10,9,7
3,train,3,5,5,6
4,foot,10,2,0,5


In [34]:
def toAdjacencyList(outranking_matrix):
    adjacencylist = {}
    for idx,i in enumerate(outranking_matrix):
#         print(idx,i)
        lst = []
        for index,k in enumerate(i):
            if k == 1:
                lst.append(index)
            adjacencylist[idx] = lst
    return adjacencylist

In [35]:
graph = toAdjacencyList(outranking_matrix)
print(graph)

{0: [], 1: [4], 2: [], 3: [0, 2], 4: []}


2.4) Draw outranking graph, and remove cycles (manually).

In [36]:
cm.PrintGraph(graph, filename="graph_part_2")

2.5) Which mode of transport are in the kernel?

# Part 3: Kernel

3.1) Given is the below outranking matrix

In [20]:
outranking_matrix =np.array(   [[0., 1., 0., 0., 0., 0., 1., 0.],
                                [0., 0., 1., 0., 1., 0., 0., 0.],
                                [0., 0., 0., 0., 0., 0., 0., 1.],
                                [0., 0., 0., 0., 1., 0., 1., 0.],
                                [0., 0., 0., 0., 0., 1., 0., 0.],
                                [0., 0., 0., 0., 0., 0., 1., 0.],
                                [0., 0., 0., 0., 0., 0., 0., 1.],
                                [0., 0., 0., 0., 0., 0., 0., 0.]])

In [21]:
graph = toAdjacencyList(outranking_matrix)

In [22]:
graph

{0: [1, 6], 1: [2, 4], 2: [7], 3: [4, 6], 4: [5], 5: [6], 6: [7], 7: []}

3.2) Display the outranking graph. Which alternatives belong to kernel?

In [23]:
cm.PrintGraph(graph, filename="graph_part_3")

3.3) In this exercise, you are asked to complete the function for constructing a kernel. Firstly, complete the below auxiliary function which reverses edges of the graph.

In [24]:
def getReverseGraph(graph):
    reverse = {}
    for i in graph:
        reverse[i] = []
    for i in graph:
        for j in graph[i]:
            reverse[j].append(i)
    return reverse

3.4) Verify the correctness: compare the below reverse_graph with the above graph.

In [25]:
reverse_graph = getReverseGraph(graph)
reverse_graph

{0: [], 1: [0], 2: [1], 3: [], 4: [1, 3], 5: [4], 6: [0, 3, 5], 7: [2, 6]}

In [26]:
cm.PrintGraph(reverse_graph, filename="reverse_graph_part_3")

3.5) Now, complete the below function for finding a graph kernel. This algorithm should proceed in the following way: <br>
1) Find non-outranked vertices. Add them to the kernel. <br> 
2) Remove vertices found in step 1 from the graph and these vertices which are directly surpassed by them.<br>
3) Repeat (go to 1) until all vertices are removed from the graph. 

In [27]:
def Kernel(graph):
    reverse_graph = getReverseGraph(graph)
    kernel = []
    step=0
    while(len(reverse_graph)>0):
        v = next(iter(reverse_graph))
        e = reverse_graph[v]
        del reverse_graph[v]
        if len(e) ==0:
            kernel.append(v)
            for v_2 in graph[v]:
                for __v in reverse_graph:
                    if v_2 in reverse_graph[__v]:
                        reverse_graph[__v].remove(v_2)
                if v_2 in reverse_graph:
                    del reverse_graph[v_2]
        else:
             reverse_graph[v]=e
    return kernel

In [28]:
Kernel(graph)

[0, 2, 3, 5]